In [54]:
from bs4 import BeautifulSoup
import requests
!pip install yelp

In [55]:
CLIENT_ID = ""
API_KEY = ""

In [56]:
with open('/Users/limenglu/Desktop/Yelp.txt','r') as f:
    count = 0
    for line in f:
        if count == 0:
            CLIENT_ID = line.strip()
        if count == 1:
            API_KEY = line.strip()
        count+=1


In [57]:
print(CLIENT_ID,API_KEY)

q6ZnB2cLEqSK0-7_JvEgrw qGhtYi8ylmU8Oh1Dol-wd39moGMSdyFl1YeYvZ-fpXPltigazwCaCiyZmuToklTxAu1rz6bkrMmenvRbMQTPan5QYl2NsBRaAYgd6KG_LjfxDY8sQXO0LGxr_WRJX3Yx


In [58]:
# API constants, you shouldn't have to change these.
API_HOST = 'https://api.yelp.com' #The API url header
SEARCH_PATH = '/v3/businesses/search' #The path for an API request to find businesses
BUSINESS_PATH = '/v3/businesses/'  # The path to get data for a single business

In [59]:
def get_restaurants(api_key,location,number=15):
    import requests
    
    #First we get the access token
    #Set up the search data dictionary
    search_data = {
    'term': "restaurant",
    'location': location.replace(' ', '+'),
    'limit': number
    }
    url = API_HOST + SEARCH_PATH
    headers = {
        'Authorization': 'Bearer %s' % api_key,
    }
    response = requests.request('GET', url, headers=headers, params=search_data).json()
    businesses = response.get('businesses')
    return businesses

In [60]:
get_restaurants(API_KEY,"San Francisco")

[{'id': 'YbUD7-Z2Be5thHaybd7ssQ',
  'alias': 'alchemist-bar-and-lounge-san-francisco',
  'name': 'Alchemist Bar & Lounge',
  'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/agYDoCkb7TJigwLVpKvQPg/o.jpg',
  'is_closed': False,
  'url': 'https://www.yelp.com/biz/alchemist-bar-and-lounge-san-francisco?adjust_creative=q6ZnB2cLEqSK0-7_JvEgrw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=q6ZnB2cLEqSK0-7_JvEgrw',
  'review_count': 423,
  'categories': [{'alias': 'lounges', 'title': 'Lounges'},
   {'alias': 'cocktailbars', 'title': 'Cocktail Bars'},
   {'alias': 'newamerican', 'title': 'American (New)'}],
  'rating': 4.0,
  'coordinates': {'latitude': 37.7791800203336,
   'longitude': -122.392967912689},
  'transactions': ['restaurant_reservation'],
  'price': '$$',
  'location': {'address1': '679 3rd St',
   'address2': None,
   'address3': '',
   'city': 'San Francisco',
   'zip_code': '94107',
   'country': 'US',
   'state': 'CA',
   'display_address': ['679 3r

In [61]:
def get_business_review(api_key,business_id):
    import json
    import requests
    business_path = BUSINESS_PATH + business_id+"/reviews"
    url = API_HOST + business_path

    headers = {
        'Authorization': 'Bearer %s' % api_key,
    }


    response = requests.request('GET', url, headers=headers).json()
   
    review_text = ''
    for review in response['reviews']:
        review_text += review['text']
    return review_text

In [62]:
get_business_review(API_KEY,'flat-top-new-york')

'I love flat top, never had a bad experience or a bad meal here. Their "airline chicken" is my go-to meal here. Have brought several friends and everyone...Cute little place on 121st and Amsterdam! Came here for brunch on Saturday with two of my friends and I\'m glad we made reservations. We came around 11:30...The server was pretty rude and not attentive at all. Will probably try out other restaurant first\n\nI got the burger and it\'s good and the portion is pretty...'

In [63]:
def get_reviews(location,number=15):

    restaurants = get_restaurants(API_KEY,location,number)

    if not restaurants:
        return None
    review_list = list()
    for restaurant in restaurants:
        restaurant_name = restaurant['name']
        restaurant_id = restaurant['id']
        review_text = get_business_review(API_KEY,restaurant_id)
        
        review_list.append((restaurant_name,review_text))
    return review_list
        



In [64]:
get_business_review(API_KEY,'flat-top-new-york')

'I love flat top, never had a bad experience or a bad meal here. Their "airline chicken" is my go-to meal here. Have brought several friends and everyone...Cute little place on 121st and Amsterdam! Came here for brunch on Saturday with two of my friends and I\'m glad we made reservations. We came around 11:30...The server was pretty rude and not attentive at all. Will probably try out other restaurant first\n\nI got the burger and it\'s good and the portion is pretty...'

In [65]:
def get_reviews(location,number=15):

    restaurants = get_restaurants(API_KEY,location,number)

    if not restaurants:
        return None
    review_list = list()
    for restaurant in restaurants:
        restaurant_name = restaurant['name']
        restaurant_id = restaurant['id']
        review_text = get_business_review(API_KEY,restaurant_id)
        
        review_list.append((restaurant_name,review_text))
    return review_list
        

In [69]:
all_snippets = get_reviews("San Francisco")

In [70]:
all_snippets

[('Alchemist Bar & Lounge',
  "My review is based on pre and post dinner drinks, so I don't have much to say in regards to the cuisine.\nSuper cozy lounge that offers an amazing selection...Wow, great food and great drinks! Came here during the pandemic; this one of the few places that's opened and have outdoor seating. It was a bit chilly at...First, this bar is OPEN in covid. They have outdoor seating that seems like it would be the restaurant / bar next door, but they serve all the alchemists..."),
 ('Fog Harbor Fish House',
  'First off, our waiter Adam; was so awesome and welcoming. It was our first time there and we are happy we went. With everything going on with COVID-19 this...I loved my time at this restaurant. Despite COVID, we had a great time. The food was really good and the seating very well adapted to the current...I made a reservation through Yelp, we were seated right away when we arrived. All of the staff were attentive and very friendly. They practice good social...

In [73]:
def get_nrc_data():
    nrc = "/Users/limenglu/Desktop/NRC-emotion-lexicon-wordlevel-alphabetized-v0.92.txt"
    count=0
    emotion_dict=dict()
    with open(nrc,'r') as f:
        all_lines = list()
        for line in f:
            if count < 46:
                count+=1
                continue
            line = line.strip().split('\t')
            if int(line[2]) == 1:
                if emotion_dict.get(line[0]):
                    emotion_dict[line[0]].append(line[1])
                else:
                    emotion_dict[line[0]] = [line[1]]
    return emotion_dict


def emotion_analyzer(text,emotion_dict=get_nrc_data()):
    #Set up the result dictionary
    emotions = {x for y in emotion_dict.values() for x in y}
    emotion_count = dict()
    for emotion in emotions:
        emotion_count[emotion] = 0

    #Analyze the text and normalize by total number of words
    total_words = len(text.split())
    for word in text.split():
        if emotion_dict.get(word):
            for emotion in emotion_dict.get(word):
                emotion_count[emotion] += 1/len(text.split())
    return emotion_count

In [74]:
print("%-12s %1s\t%1s %1s %1s %1s   %1s %1s %1s %1s"%(
        "restaurant","fear","trust","negative","positive","joy","disgust","anticip",
        "sadness","surprise"))
        
for snippet in all_snippets:
    text = snippet[1]
    result = emotion_analyzer(text)
    print("%-12s %1.2f\t%1.2f\t%1.2f\t%1.2f\t%1.2f\t%1.2f\t%1.2f\t%1.2f\t%1.2f"%(
        snippet[0][0:10],result['fear'],result['trust'],
          result['negative'],result['positive'],result['joy'],result['disgust'],
          result['anticipation'],result['sadness'],result['surprise']))


restaurant   fear	trust negative positive joy   disgust anticip sadness surprise
Alchemist    0.00	0.02	0.03	0.02	0.01	0.00	0.00	0.00	0.00
Fog Harbor   0.00	0.06	0.00	0.07	0.05	0.00	0.06	0.00	0.02
The Detour   0.01	0.04	0.01	0.05	0.04	0.01	0.04	0.01	0.03
Chicken as   0.03	0.03	0.00	0.04	0.03	0.00	0.01	0.00	0.00
Gary Danko   0.01	0.04	0.01	0.06	0.04	0.00	0.04	0.00	0.01
Burma Supe   0.00	0.03	0.00	0.03	0.00	0.00	0.00	0.00	0.00
Sotto Mare   0.01	0.04	0.03	0.06	0.05	0.00	0.05	0.01	0.04
Hog Island   0.01	0.01	0.04	0.01	0.01	0.01	0.05	0.01	0.00
Tacos El P   0.02	0.01	0.04	0.01	0.01	0.00	0.00	0.01	0.02
Farmhouse    0.00	0.01	0.00	0.04	0.03	0.00	0.01	0.00	0.00
Kokkari Es   0.00	0.05	0.00	0.05	0.03	0.00	0.00	0.00	0.00
Daeho Kalb   0.00	0.02	0.02	0.03	0.01	0.01	0.02	0.01	0.00
Coqueta      0.01	0.05	0.01	0.04	0.01	0.00	0.02	0.01	0.00
San Tung     0.00	0.01	0.01	0.06	0.02	0.00	0.05	0.00	0.02
Son & Gard   0.00	0.02	0.04	0.06	0.02	0.00	0.01	0.01	0.01


In [75]:
def comparative_emotion_analyzer(text_tuples,object_name="Restaurant",print_output=False):
    if print_output:
        print("%-20s %1s\t%1s %1s %1s %1s   %1s %1s %1s %1s"%(object_name,
                                                              "fear","trust","negative","positive",
                                                              "joy","disgust","anticip", "sadness",
                                                              "surprise"))
    import pandas as pd
    df = pd.DataFrame(columns=[object_name,'Fear','Trust','Negative',
                           'Positive','Joy','Disgust','Anticipation',
                           'Sadness','Surprise'],)
    df.set_index(object_name,inplace=True)
    
    output = df    
    for text_tuple in text_tuples:
        text = text_tuple[1] 
        result = emotion_analyzer(text)
        if print_output:
            print("%-20s %1.2f\t%1.2f\t%1.2f\t%1.2f\t%1.2f\t%1.2f\t%1.2f\t%1.2f\t%1.2f"%(
                text_tuple[1][0:20],result['fear'],result['trust'],
                  result['negative'],result['positive'],result['joy'],result['disgust'],
                  result['anticipation'],result['sadness'],result['surprise']))
        df.loc[text_tuple[0]] = [result['fear'],result['trust'],
                  result['negative'],result['positive'],result['joy'],result['disgust'],
                  result['anticipation'],result['sadness'],result['surprise']]
    return output       
comparative_emotion_analyzer(all_snippets)

,Fear,Trust,Negative,Positive,Joy,Disgust,Anticipation,Sadness,Surprise
Restaurant,,,,,,,,,
Alchemist Bar & Lounge,0.000000,0.023256,0.034884,0.023256,0.011628,0.000000,0.000000,0.000000,0.000000
Fog Harbor Fish House,0.000000,0.060976,0.000000,0.073171,0.048780,0.000000,0.060976,0.000000,0.024390
The Detour,0.012658,0.037975,0.012658,0.050633,0.037975,0.012658,0.037975,0.012658,0.025316
Chicken as Cluck,0.026316,0.026316,0.000000,0.039474,0.026316,0.000000,0.013158,0.000000,0.000000
Gary Danko,0.012195,0.036585,0.012195,0.060976,0.036585,0.000000,0.036585,0.000000,0.012195
Burma Superstar,0.000000,0.028169,0.000000,0.028169,0.000000,0.000000,0.000000,0.000000,0.000000
Sotto Mare Oysteria & Seafood,0.012658,0.037975,0.025316,0.063291,0.050633,0.000000,0.050633,0.012658,0.037975
Hog Island Oyster Co,0.011905,0.011905,0.035714,0.011905,0.011905,0.011905,0.047619,0.011905,0.000000
Tacos El Patrón,0.023529,0.011765,0.035294,0.011765,0.011765,0.000000,0.000000,0.011765,0.023529


In [76]:
def analyze_nearby_restaurants(address,number=15):
    snippets = get_reviews(address,number)
    return comparative_emotion_analyzer(snippets)
 
analyze_nearby_restaurants("Bay Area",15)

,Fear,Trust,Negative,Positive,Joy,Disgust,Anticipation,Sadness,Surprise
Restaurant,,,,,,,,,
World Famous Hotboys,0.059524,0.000000,0.023810,0.000000,0.000000,0.000000,0.023810,0.000000,0.000000
Daughter Thai Kitchen,0.000000,0.000000,0.000000,0.058140,0.011628,0.000000,0.023256,0.000000,0.000000
Ruby Q Smoke Fusion,0.011765,0.023529,0.000000,0.047059,0.035294,0.000000,0.000000,0.000000,0.000000
Pomella,0.000000,0.039474,0.000000,0.052632,0.039474,0.000000,0.013158,0.000000,0.013158
Gus’s World Famous Fried Chicken,0.000000,0.022222,0.011111,0.033333,0.022222,0.011111,0.033333,0.000000,0.011111
Habibi’s Kitchen,0.000000,0.037037,0.000000,0.074074,0.049383,0.000000,0.037037,0.000000,0.012346
Oakland Street Food Co,0.000000,0.023256,0.011628,0.069767,0.034884,0.000000,0.011628,0.011628,0.000000
Perle Wine Bar,0.000000,0.058140,0.000000,0.093023,0.046512,0.000000,0.000000,0.000000,0.000000
Kokolo Donburi & Grill,0.000000,0.023810,0.011905,0.023810,0.000000,0.000000,0.023810,0.000000,0.000000
